<a href="https://colab.research.google.com/github/aarohishaiva/Phonepay_project_2/blob/main/Project_2_Phone_pe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install mysql-connector-python streamlit plotly pillow streamlit-option-menu
!pip install sqlalchemy

# Import libraries
import pandas as pd
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.4/823.4 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/PhonePe/pulse.git

Cloning into 'pulse'...
remote: Enumerating objects: 15229, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 15229 (delta 3), reused 1 (delta 1), pack-reused 15202
Receiving objects: 100% (15229/15229), 21.95 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (7745/7745), done.
Updating files: 100% (7921/7921), done.


In [3]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os
import json


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path1 = "/content/pulse/data/aggregated/transaction/country/india/state/"
agg_trans_list = os.listdir(path1)

columns1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}
for state in agg_trans_list:
    all_state = path1 + state + "/"
    agg_year_list = os.listdir(all_state)

    for year in agg_year_list:
        all_year = all_state + year + "/"
        agg_file_list = os.listdir(all_year)

        for file in agg_file_list:
            cur_file = all_year + file
            data = open(cur_file, 'r')
            A = json.load(data)

            for i in A['data']['transactionData']:
                name = i['name']
                count = i['paymentInstruments'][0]['count']
                amount = i['paymentInstruments'][0]['amount']
                columns1['Transaction_type'].append(name)
                columns1['Transaction_count'].append(count)
                columns1['Transaction_amount'].append(amount)
                columns1['State'].append(state)
                columns1['Year'].append(year)
                columns1['Quarter'].append(int(file.strip('.json')))

df_agg_trans = pd.DataFrame(columns1)

In [ ]:
df_agg_trans.shape

(4494, 6)

In [ ]:
path2 = "/content/pulse/data/aggregated/user/country/india/state/"

agg_user_list = os.listdir(path2)

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Brands': [], 'Count': [],
            'Percentage': []}
for state in agg_user_list:
    cur_state = path2 + state + "/"
    agg_year_list = os.listdir(cur_state)

    for year in agg_year_list:
        cur_year = cur_state + year + "/"
        agg_file_list = os.listdir(cur_year)

        for file in agg_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            B = json.load(data)
            try:
                for i in B["data"]["usersByDevice"]:
                    brand_name = i["brand"]
                    counts = i["count"]
                    percents = i["percentage"]
                    columns2["Brands"].append(brand_name)
                    columns2["Count"].append(counts)
                    columns2["Percentage"].append(percents)
                    columns2["State"].append(state)
                    columns2["Year"].append(year)
                    columns2["Quarter"].append(int(file.strip('.json')))
            except:
                pass
df_agg_user = pd.DataFrame(columns2)
df_agg_user

,State,Year,Quarter,Brands,Count,Percentage
0,himachal-pradesh,2018,1,Xiaomi,63002,0.277087
1,himachal-pradesh,2018,1,Samsung,36283,0.159575
2,himachal-pradesh,2018,1,Vivo,20480,0.090072
3,himachal-pradesh,2018,1,Oppo,20074,0.088287
4,himachal-pradesh,2018,1,Realme,10722,0.047156
...,...,...,...,...,...,...
6727,assam,2021,2,Asus,45364,0.012542
6728,assam,2021,2,Motorola,42821,0.011839
6729,assam,2021,2,Apple,41622,0.011507
6730,assam,2021,2,Gionee,41574,0.011494


In [39]:
path3 = "/content/pulse/data/map/transaction/hover/country/india/state/"  # Add a trailing slash

map_trans_list = os.listdir(path3)

columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [],
            'Amount': []}

for state in map_trans_list:
    cur_state = path3 + state + "/"  # Now correctly constructs the path with the trailing slash
    map_year_list = os.listdir(cur_state)

    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)

        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            C = json.load(data)

            for i in C["data"]["hoverDataList"]:
                district = i["name"]
                count = i["metric"][0]["count"]
                amount = i["metric"][0]["amount"]
                columns3["District"].append(district)
                columns3["Count"].append(count)
                columns3["Amount"].append(amount)
                columns3['State'].append(state)
                columns3['Year'].append(year)
                columns3['Quarter'].append(int(file.strip('.json')))

df_map_trans = pd.DataFrame(columns3)
df_map_trans

,State,Year,Quarter,District,Count,Amount
0,himachal-pradesh,2018,1,solan district,118881,1.468736e+08
1,himachal-pradesh,2018,1,sirmaur district,37721,4.357953e+07
2,himachal-pradesh,2018,1,shimla district,54161,8.437553e+07
3,himachal-pradesh,2018,1,bilaspur district,21880,2.021559e+07
4,himachal-pradesh,2018,1,hamirpur district,32709,3.750625e+07
...,...,...,...,...,...,...
18291,assam,2024,1,hailakandi district,4160640,6.161516e+09
18292,assam,2024,1,barpeta district,18751526,2.932517e+10
18293,assam,2024,1,goalpara district,10539700,1.660675e+10
18294,assam,2024,1,biswanath district,2719000,3.417757e+09


In [45]:
path4 = "/content/pulse/data/map/user/hover/country/india/state/" # Add trailing slash to ensure correct path construction

map_user_list = os.listdir(path4)

columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}

for state in map_user_list:
    cur_state = path4 + state + "/"
    map_year_list = os.listdir(cur_state)

    for year in map_year_list:
        cur_year = cur_state + year + "/"
        map_file_list = os.listdir(cur_year)

        for file in map_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            D = json.load(data)

            for i in D["data"]["hoverData"].items():
                district = i[0]
                registereduser = i[1]["registeredUsers"]
                appOpens = i[1]['appOpens']
                columns4["District"].append(district)
                columns4["RegisteredUser"].append(registereduser)
                columns4["AppOpens"].append(appOpens)
                columns4['State'].append(state)
                columns4['Year'].append(year)
                columns4['Quarter'].append(int(file.strip('.json')))

df_map_user = pd.DataFrame(columns4)
df_map_user

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,himachal-pradesh,2018,1,solan district,50035,0
1,himachal-pradesh,2018,1,sirmaur district,15880,0
2,himachal-pradesh,2018,1,shimla district,28214,0
3,himachal-pradesh,2018,1,bilaspur district,11975,0
4,himachal-pradesh,2018,1,hamirpur district,15614,0
...,...,...,...,...,...,...
18295,assam,2024,1,hailakandi district,112961,24561651
18296,assam,2024,1,barpeta district,348707,47535673
18297,assam,2024,1,goalpara district,182003,36199591
18298,assam,2024,1,biswanath district,105545,44686035


In [46]:
path5 = "/content/pulse/data/top/transaction/country/india/state/"

top_trans_list = os.listdir(path5)
columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
            'Transaction_amount': []}

for state in top_trans_list:
    cur_state = path5 + state + "/"
    top_year_list = os.listdir(cur_state)

    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)

        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            E = json.load(data)

            for i in E['data']['pincodes']:
                name = i['entityName']
                count = i['metric']['count']
                amount = i['metric']['amount']
                columns5['Pincode'].append(name)
                columns5['Transaction_count'].append(count)
                columns5['Transaction_amount'].append(amount)
                columns5['State'].append(state)
                columns5['Year'].append(year)
                columns5['Quarter'].append(int(file.strip('.json')))
df_top_trans = pd.DataFrame(columns5)
df_top_trans

,State,Year,Quarter,Pincode,Transaction_count,Transaction_amount
0,himachal-pradesh,2018,1,173205,50457,5.895455e+07
1,himachal-pradesh,2018,1,176323,28116,4.754874e+07
2,himachal-pradesh,2018,1,174101,23993,2.685392e+07
3,himachal-pradesh,2018,1,174303,19122,2.743764e+07
4,himachal-pradesh,2018,1,173025,16138,2.246522e+07
...,...,...,...,...,...,...
8919,assam,2024,1,784115,2353440,3.803045e+09
8920,assam,2024,1,781319,2339414,3.333857e+09
8921,assam,2024,1,783384,2205735,3.114914e+09
8922,assam,2024,1,781038,2201708,2.428073e+09


In [47]:
path6 = "/content/pulse/data/top/user/country/india/state/"
top_user_list = os.listdir(path6)
columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
            'RegisteredUsers': []}

for state in top_user_list:
    cur_state = path6 + state + "/"
    top_year_list = os.listdir(cur_state)

    for year in top_year_list:
        cur_year = cur_state + year + "/"
        top_file_list = os.listdir(cur_year)

        for file in top_file_list:
            cur_file = cur_year + file
            data = open(cur_file, 'r')
            F = json.load(data)

            for i in F['data']['pincodes']:
                name = i['name']
                registeredUsers = i['registeredUsers']
                columns6['Pincode'].append(name)
                columns6['RegisteredUsers'].append(registeredUsers)
                columns6['State'].append(state)
                columns6['Year'].append(year)
                columns6['Quarter'].append(int(file.strip('.json')))
df_top_user = pd.DataFrame(columns6)
df_top_user

,State,Year,Quarter,Pincode,RegisteredUsers
0,himachal-pradesh,2018,1,173205,21409
1,himachal-pradesh,2018,1,174101,10157
2,himachal-pradesh,2018,1,173025,6327
3,himachal-pradesh,2018,1,175131,4677
4,himachal-pradesh,2018,1,171001,4527
...,...,...,...,...,...
8920,assam,2024,1,781011,52141
8921,assam,2024,1,785001,50527
8922,assam,2024,1,786001,49628
8923,assam,2024,1,782435,47562


In [48]:
!pip install pandas sqlalchemy streamlit plotly ngrok


In [94]:
import pandas as pd
import os
import json
from sqlalchemy import create_engine

# Define the path to the directory containing the data
path1 = "/content/pulse/data/aggregated/transaction/country/india/state/"
path2 = "/content/pulse/data/aggregated/user/country/india/state/"
path3 = "/content/pulse/data/map/transaction/hover/country/india/state/"
path4 = "/content/pulse/data/map/user/hover/country/india/state/"
path5 = "/content/pulse/data/top/transaction/country/india/state/"
path6 = "/content/pulse/data/top/user/country/india/state/"

# List all directories (states) in the path
agg_trans_list = os.listdir(path1)
agg_user_list = os.listdir(path2)
map_trans_list = os.listdir(path3)
map_user_list = os.listdir(path4)
top_trans_list = os.listdir(path5)
top_user_list = os.listdir(path6)

# Initialize a dictionary to store the data
columns1 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'Transaction_count': [],
            'Transaction_amount': []}

columns2 = {'State': [], 'Year': [], 'Quarter': [], 'Transaction_type': [], 'RegisteredUsers': []}

columns3 = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Count': [], 'Amount': []}

columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
            "RegisteredUser": [], "AppOpens": []}

columns5 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [], 'Transaction_count': [],
            'Transaction_amount': []}
columns6 = {'State': [], 'Year': [], 'Quarter': [], 'Pincode': [],
            'RegisteredUsers': []}

# Iterate over each state directory
for state in top_user_list:
    cur_state = os.path.join(path6, state)

    # Check if the path is a directory
    if os.path.isdir(cur_state):
        top_year_list = os.listdir(cur_state)

        for year in top_year_list:
            cur_year = os.path.join(cur_state, year)

            # Check if the path is a directory
            if os.path.isdir(cur_year):
                top_file_list = os.listdir(cur_year)

                for file in top_file_list:
                    cur_file = os.path.join(cur_year, file)

                    # Check if the path is a file and ends with .json
                    if os.path.isfile(cur_file) and file.endswith('.json'):
                        with open(cur_file, 'r') as data:
                            try:
                                F = json.load(data)
                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON from file {cur_file}: {e}")
                                continue

                            if F and "data" in F and "pincodes" in F["data"]:
                                for i in F["data"]["pincodes"]:
                                    name = i.get("name")
                                    registeredUsers = i.get("registeredUsers")

                                    columns6['Pincode'].append(name)
                                    columns6['RegisteredUsers'].append(registeredUsers)
                                    columns6['State'].append(state)
                                    columns6['Year'].append(year)
                                    columns6['Quarter'].append(int(file.strip('.json')))

# Create a DataFrame from the dictionary
df_top_user = pd.DataFrame(columns6)

# Create an SQLite database and insert the data
engine = create_engine('sqlite:///phonepe_data.db')
df_top_user.to_sql('phonepe_data', engine, if_exists='replace', index=False)

print("Data inserted into SQLite database successfully!")


Data inserted into SQLite database successfully!


In [96]:
%%writefile app.py
import streamlit as st
from PIL import Image
import requests
import os
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import json

# Set page configuration
st.set_page_config(layout="wide")
st.title("PHONEPE DATA VISUALIZATION AND EXPLORATION")

# Sidebar menu
with st.sidebar:
    select = st.selectbox("Main Menu", ["HOME", "DATA EXPLORATION", "TOP CHARTS"])

# HOME Page
if select == "HOME":
    col1, col2 = st.columns(2)

    with col1:
        st.header("PHONEPE")
        st.subheader("INDIA'S BEST TRANSACTION APP")
        st.markdown("PhonePe is an Indian digital payments and financial technology company.")
        st.write("****FEATURES****")
        st.write("****Credit & Debit card linking****")
        st.write("****Bank Balance check****")
        st.write("****Money Storage****")
        st.write("****PIN Authorization****")
        st.download_button("DOWNLOAD THE APP NOW", "https://www.phonepe.com/app-download/")

    with col2:
        image_url = "https://example.com/image.jpg"  # Replace with the correct image URL
        image_path = "image.jpg"
        # Download the image file
        if not os.path.exists(image_path):
            response = requests.get(image_url)
            with open(image_path, "wb") as f:
                f.write(response.content)
        try:
            image = Image.open(image_path)
            st.image(image, caption="PhonePe App", use_column_width=True)
        except IOError:
            st.error("Failed to load image. Please check the file format and try again.")

    col3, col4 = st.columns(2)

    with col3:
        st.write("****Easy Transactions****")
        st.write("****One App For All Your Payments****")
        st.write("****Your Bank Account Is All You Need****")
        st.write("****Multiple Payment Modes****")
        st.write("****PhonePe Merchants****")

    with col4:
        st.write("****Multiple Ways To Pay****")
        st.write("****1. Direct Transfer & More****")
        st.write("****2. QR Code****")
        st.write("****Earn Great Rewards****")
        st.write("****No Wallet Top-Up Required****")
        st.write("****Pay Directly From Any Bank To Any Bank A/C****")
        st.write("****Instantly & Free****")

# DATA EXPLORATION Page
if select == "DATA EXPLORATION":
    tab1, tab2, tab3 = st.tabs(["Aggregated Analysis", "Map Analysis", "Top Analysis"])

    with tab1:
        method = st.radio("Select The Method", ["Insurance Analysis", "Transaction Analysis", "User Analysis"])

        if method == "Insurance Analysis":
            st.write("Insurance Analysis content goes here.")
            # Sample Data
            st.write(pd.DataFrame({
                "Insurance Type": ["Type A", "Type B", "Type C"],
                "Count": [120, 150, 80]
            }))

        elif method == "Transaction Analysis":
            st.write("Transaction Analysis content goes here.")
            # Sample Data
            st.write(pd.DataFrame({
                "Transaction ID": [1, 2, 3],
                "Amount": [1000, 2000, 1500]
            }))

        elif method == "User Analysis":
            st.write("User Analysis content goes here.")
            # Sample Data
            st.write(pd.DataFrame({
                "User ID": [101, 102, 103],
                "Registration Date": ["2024-01-01", "2024-02-01", "2024-03-01"]
            }))

    with tab2:
        method_2 = st.radio("Select The Method", ["Map Insurance", "Map Transaction", "Map User"])

        if method_2 == "Map Insurance":
            st.write("Map Insurance content goes here.")
            # Sample Map Placeholder
            st.write("Map visualization for Insurance")

        elif method_2 == "Map Transaction":
            st.write("Map Transaction content goes here.")
            # Sample Map Placeholder
            st.write("Map visualization for Transactions")

        elif method_2 == "Map User":
            st.write("Map User content goes here.")
            # Sample Map Placeholder
            st.write("Map visualization for Users")

    with tab3:
        method_3 = st.radio("Select The Method", ["Top Insurance", "Top Transaction", "Top User"])

        if method_3 == "Top Insurance":
            st.write("Top Insurance content goes here.")
            # Sample Data
            st.write(pd.DataFrame({
                "Insurance Type": ["Type A", "Type B"],
                "Top Count": [50, 30]
            }))

        elif method_3 == "Top Transaction":
            st.write("Top Transaction content goes here.")
            # Sample Data
            st.write(pd.DataFrame({
                "Transaction ID": [1, 2],
                "Top Amount": [5000, 4000]
            }))

        elif method_3 == "Top User":
            st.write("Top User content goes here.")
            # Sample Data
            st.write(pd.DataFrame({
                "User ID": [101, 102],
                "Top Transactions": [10, 8]
            }))

# TOP CHARTS Page
if select == "TOP CHARTS":
    question = st.selectbox("Select the Question", [
        "1. Transaction Amount and Count of Aggregated Insurance",
        "2. Transaction Amount and Count of Map Insurance",
        "3. Transaction Amount and Count of Top Insurance",
        "4. Transaction Amount and Count of Aggregated Transaction",
        "5. Transaction Amount and Count of Map Transaction",
        "6. Transaction Amount and Count of Top Transaction",
        "7. Transaction Count of Aggregated User",
        "8. Registered users of Map User",
        "9. App opens of Map User",
        "10. Registered users of Top User",
    ])

    if question in ["1. Transaction Amount and Count of Aggregated Insurance",
                    "2. Transaction Amount and Count of Map Insurance",
                    "3. Transaction Amount and Count of Top Insurance",
                    "4. Transaction Amount and Count of Aggregated Transaction",
                    "5. Transaction Amount and Count of Map Transaction",
                    "6. Transaction Amount and Count of Top Transaction"]:
        st.subheader("TRANSACTION AMOUNT")
        top_chart_transaction_amount(question.split()[0].lower())

        st.subheader("TRANSACTION COUNT")
        top_chart_transaction_count(question.split()[0].lower())

    elif question == "7. Transaction Count of Aggregated User":
        st.subheader("TRANSACTION COUNT")
        top_chart_transaction_count("aggregated_user")

    elif question == "8. Registered users of Map User":
        states = ["State A", "State B"]  # Replace with dynamic state data
        state = st.selectbox("Select the State", states)
        st.subheader("REGISTERED USERS")
        top_chart_registered_user("map_user", state)

    elif question == "9. App opens of Map User":
        states = ["State A", "State B"]  # Replace with dynamic state data
        state = st.selectbox("Select the State", states)
        st.subheader("APPOPENS")
        top_chart_appopens("map_user", state)

    elif question == "10. Registered users of Top User":
        st.subheader("REGISTERED USERS")
        top_chart_registered_users("top_user")

# Placeholder functions
@st.cache_data
def top_chart_transaction_amount(method):
    # Sample Chart Placeholder
    st.write(f"Displaying top chart transaction amount for {method}")

@st.cache_data
def top_chart_transaction_count(method):
    # Sample Chart Placeholder
    st.write(f"Displaying top chart transaction count for {method}")

@st.cache_data
def top_chart_registered_user(method, state):
    # Sample Data Table Placeholder
    st.write(f"Displaying registered users for {method} in state {state}")
    st.write(pd.DataFrame({
        "Pincode": ["12345", "67890"],
        "Registered Users": [100, 200]
    }))

@st.cache_data
def top_chart_appopens(method, state):
    # Sample Data Table Placeholder
    st.write(f"Displaying app opens for {method} in state {state}")
    st.write(pd.DataFrame({
        "Pincode": ["12345", "67890"],
        "App Opens": [300, 400]
    }))

@st.cache_data
def top_chart_registered_users(method):
    # Sample Data Table Placeholder
    st.write(f"Displaying registered users for {method}")
    st.write(pd.DataFrame({
        "State": ["State A", "State B"],
        "Registered Users": [500, 600]
    }))

@st.cache_data
def load_map_data():
    # Load map data
    df = pd.read_csv("https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/active_cases_2020-07-17_0800.csv")
    return df

@st.cache_data
def generate_choropleth(df):
    fig = px.choropleth(
        df,
        geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
        featureidkey='properties.ST_NM',
        locations='state',
        color='active cases',
        color_continuous_scale='reds'
    )
    fig.update_geos(fitbounds="locations", visible=False)
    return fig

# Load map data and generate choropleth plot
df = load_map_data()
fig = generate_choropleth(df)

st.plotly_chart(fig)


Overwriting app.py


In [97]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/active_cases_2020-07-17_0800.csv")

fig = px.choropleth(
    df,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='state',
    color='active cases',
    color_continuous_scale='reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [98]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.113.143:8501

  Stopping...
  Stopping...


In [99]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "2iT1S8jORWP8AFT2gipGnOs9RQe_325YotmHc3irE4pTUSuSp"

In [100]:
# Install ngrok
!pip install pyngrok

In [101]:
# Import ngrok
from pyngrok import ngrok

# Authenticate ngrok with your token
ngrok.set_auth_token("2iT1S8jORWP8AFT2gipGnOs9RQe_325YotmHc3irE4pTUSuSp")

# Start ngrok to tunnel the Streamlit app
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run app.py &


Streamlit app is live at: NgrokTunnel: "https://5cc4-34-83-113-143.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.113.143:8501

  Stopping...
